In [ ]:
import networkx as nx
from sage.graphs.graph_generators_pyx import RandomGNP
import pulp

In [ ]:
# generator can either be erdos_renyi_graph or RandomGNP
def generateGraph(n, density, generator = nx.erdos_renyi_graph):
    g = generator(n, density)
    
    if type(g) is not sage.graphs.graph.Graph:
        g = Graph(g)
        
    return g


def computeMDS(graph):
    dominating_set = graph.dominating_set()
    return dominating_set


def verifyMDS(graph, dominating_set):
    nxMDS = nx.algorithms.dominating.is_dominating_set(graph.networkx_graph(), dominating_set)
    sageMDS = g.is_dominating(dominating_set)
    return nxMDS and sageMDS

In [ ]:
def generateMDSSolutions(graph, num_solutions = 3, show_messages=True):
    g = graph.networkx_graph()
    
    prob = pulp.LpProblem("Minimum Dominating Set", pulp.LpMinimize)
    y = pulp.LpVariable.dicts("y", g.nodes(), cat=pulp.LpBinary)

    prob += pulp.lpSum(y)
    for v in g.nodes():
        prob += (y[v] + pulp.lpSum([y[u] for u in g.neighbors(v)]) >= 1)
        
    solutions = []

    for i in range(num_solutions):
        prob.solve(pulp.apis.PULP_CBC_CMD(msg=show_messages))
        
        selected_vars = []
        for var in y:
             if y[var].value() != 0:
                selected_vars.append(var)
        
        solutions.append(selected_vars)
        
        # Add a new constraint to prevent same solution from being found again
        prob += (pulp.lpSum([y[var] for var in selected_vars]) <= len(selected_vars) - 1)
        
        if pulp.LpStatus[prob.status] != 'Optimal':
            break
            
    return solutions

In [ ]:
graph = generateGraph(100, 0.3)
solutions = generateMDSSolutions(graph, num_solutions = 5)


In [ ]:
print(solutions)

In [ ]:
print(computeMDS(graph))